<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/Asphaltene_Modeling_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neqsim -q

## 1. Introduction to Asphaltenes

**Asphaltenes** are the heaviest, most polar fraction of crude oil:
- Soluble in aromatic solvents (toluene)
- Insoluble in paraffinic solvents (n-heptane)

### Production Problems
| Problem | Impact |
|---------|--------|
| Wellbore plugging | Reduced production |
| Pipeline deposition | Flow restriction |
| Equipment fouling | Reduced efficiency |

### Typical Properties
| Property | Range |
|----------|-------|
| Molecular Weight | 500-10,000 g/mol |
| H/C Ratio | 0.9-1.2 |
| Aromaticity | 40-60% |

## 2. Setup NeqSim with Direct Java Access

In [ ]:
# Install neqsim if needed
# !pip install neqsim

import neqsim
from neqsim import jneqsim
import matplotlib.pyplot as plt
import numpy as np

# Direct Java class access
SystemSrkEos = jneqsim.thermo.system.SystemSrkEos
SystemSrkCPAstatoil = jneqsim.thermo.system.SystemSrkCPAstatoil
ThermodynamicOperations = jneqsim.thermodynamicoperations.ThermodynamicOperations
PedersenAsphalteneCharacterization = jneqsim.thermo.characterization.PedersenAsphalteneCharacterization

print(f"NeqSim version: {neqsim.__version__}")

## 3. SARA Analysis and Stability Indicators

SARA fractionation separates crude oil into:
- **S**aturates: Alkanes and cycloalkanes
- **A**romatics: Aromatic hydrocarbons
- **R**esins: Polar heteroatom compounds
- **A**sphaltenes: Heaviest polar fraction

### Colloidal Instability Index (CII)
$$CII = \frac{Saturates + Asphaltenes}{Aromatics + Resins}$$

| CII | Stability |
|-----|----------|
| < 0.7 | Stable |
| 0.7-0.9 | Metastable |
| > 0.9 | Unstable |

In [ ]:
# SARA Analysis Example
def calculate_stability_indicators(saturates, aromatics, resins, asphaltenes):
    """Calculate CII and R/A ratio from SARA fractions."""
    cii = (saturates + asphaltenes) / (aromatics + resins)
    ra_ratio = resins / asphaltenes if asphaltenes > 0 else float('inf')

    # Stability assessment
    if cii < 0.7:
        stability = "STABLE"
    elif cii < 0.9:
        stability = "METASTABLE"
    else:
        stability = "UNSTABLE"

    return cii, ra_ratio, stability

# Example: Light Arabian crude
cii, ra, status = calculate_stability_indicators(0.60, 0.25, 0.10, 0.05)
print(f"CII: {cii:.2f}")
print(f"R/A Ratio: {ra:.1f}")
print(f"Status: {status}")

## 4. De Boer Empirical Screening

Fast screening based on field correlations (De Boer et al., 1995).

### Key Parameters
- **Undersaturation**: $\Delta P = P_{reservoir} - P_{bubble}$
- **In-situ density**: Live oil density at reservoir conditions

### Risk Zones
| Zone | Risk Level |
|------|------------|
| High ΔP + Low ρ | SEVERE |
| Low ΔP + High ρ | NO PROBLEM |

In [ ]:
def de_boer_screening(p_reservoir, p_bubble, density):
    """De Boer asphaltene screening correlation."""
    delta_p = p_reservoir - p_bubble

    # Empirical boundary functions
    slight = 75.0 + 0.15 * (density - 700)
    moderate = 150.0 + 0.20 * (density - 700)
    severe = 250.0 + 0.25 * (density - 700)

    if delta_p < slight:
        return "NO_PROBLEM", delta_p / slight
    elif delta_p < moderate:
        return "SLIGHT_PROBLEM", delta_p / moderate
    elif delta_p < severe:
        return "MODERATE_PROBLEM", delta_p / severe
    else:
        return "SEVERE_PROBLEM", 1.0

# Example screening
reservoirs = [
    {"name": "Field A", "p_res": 350, "p_bub": 150, "density": 720},
    {"name": "Field B", "p_res": 280, "p_bub": 180, "density": 780},
    {"name": "Field C", "p_res": 400, "p_bub": 120, "density": 690},
]

print("De Boer Screening Results")
print("-" * 50)
for r in reservoirs:
    risk, index = de_boer_screening(r["p_res"], r["p_bub"], r["density"])
    print(f"{r['name']}: ΔP={r['p_res']-r['p_bub']} bar, ρ={r['density']} kg/m³ → {risk}")

## 5. Pedersen Classical Cubic EOS Method

Models asphaltene as a heavy pseudo-component using SRK/PR EOS.

**Key Insight**: Precipitation = liquid-liquid phase split (not solid)

### Critical Property Correlations
$$T_c = a_0 + a_1 \ln(M) + a_2 M + \frac{a_3}{M}$$
$$\ln(P_c) = b_0 + b_1 \rho^{0.25} + \frac{b_2}{M} + \frac{b_3}{M^2}$$

In [ ]:
# Create SRK system with Pedersen asphaltene
fluid = SystemSrkEos(373.15, 200.0)  # 100°C, 200 bar
fluid.addComponent("methane", 0.40)
fluid.addComponent("ethane", 0.05)
fluid.addComponent("propane", 0.05)
fluid.addComponent("n-heptane", 0.35)
fluid.addComponent("nC10", 0.10)

# Characterize and add asphaltene
asph_char = PedersenAsphalteneCharacterization()
asph_char.setAsphalteneMW(750.0)      # g/mol
asph_char.setAsphalteneDensity(1.10)  # g/cm³
asph_char.addAsphalteneToSystem(fluid, 0.05)

# Initialize
fluid.setMixingRule("classic")
fluid.init(0)

# Print characterization
print(asph_char.toString())

In [ ]:
# Flash calculation
ops = ThermodynamicOperations(fluid)
ops.TPflash()

print(f"Temperature: {fluid.getTemperature() - 273.15:.1f} °C")
print(f"Pressure: {fluid.getPressure():.1f} bar")
print(f"Number of phases: {fluid.getNumberOfPhases()}")
print(f"Gas fraction: {fluid.getBeta():.4f}")

In [ ]:
# Pressure depletion study
pressures = [300, 250, 200, 150, 100, 50]
phases = []
gas_fractions = []

for p in pressures:
    fluid.setPressure(p)
    ops.TPflash()
    phases.append(fluid.getNumberOfPhases())
    gas_fractions.append(fluid.getBeta())

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(pressures, phases, 'bo-')
ax1.set_xlabel('Pressure [bar]')
ax1.set_ylabel('Number of Phases')
ax1.set_title('Phase Behavior During Depletion')
ax1.grid(True)

ax2.plot(pressures, gas_fractions, 'ro-')
ax2.set_xlabel('Pressure [bar]')
ax2.set_ylabel('Gas Fraction')
ax2.set_title('Gas Evolution')
ax2.grid(True)

plt.tight_layout()
plt.show()

## 6. Tuning Pedersen Parameters

Adjust critical property multipliers to match experimental onset pressure.

In [ ]:
# Effect of tuning parameters
mw_values = [600, 750, 1000, 1500]
tc_values = []
pc_values = []
omega_values = []

for mw in mw_values:
    char = PedersenAsphalteneCharacterization()
    char.setAsphalteneMW(mw)
    char.setAsphalteneDensity(1.10)
    char.characterize()

    tc_values.append(char.getCriticalTemperature())
    pc_values.append(char.getCriticalPressure())
    omega_values.append(char.getAcentricFactor())

# Plot MW sensitivity
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(mw_values, tc_values, 'bo-')
axes[0].set_xlabel('MW [g/mol]')
axes[0].set_ylabel('Tc [K]')
axes[0].set_title('Critical Temperature')
axes[0].grid(True)

axes[1].plot(mw_values, pc_values, 'ro-')
axes[1].set_xlabel('MW [g/mol]')
axes[1].set_ylabel('Pc [bar]')
axes[1].set_title('Critical Pressure')
axes[1].grid(True)

axes[2].plot(mw_values, omega_values, 'go-')
axes[2].set_xlabel('MW [g/mol]')
axes[2].set_ylabel('ω [-]')
axes[2].set_title('Acentric Factor')
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Apply tuning multipliers
tuned_char = PedersenAsphalteneCharacterization()
tuned_char.setAsphalteneMW(750.0)
tuned_char.setAsphalteneDensity(1.10)

# Base properties
tuned_char.characterize()
base_tc = tuned_char.getCriticalTemperature()
base_pc = tuned_char.getCriticalPressure()

# Apply tuning
tuned_char.setTcMultiplier(1.05)   # +5% Tc
tuned_char.setPcMultiplier(0.95)   # -5% Pc
tuned_char.setOmegaMultiplier(1.02)  # +2% omega
tuned_char.characterize()

print(f"Base Tc: {base_tc:.1f} K → Tuned: {tuned_char.getCriticalTemperature():.1f} K")
print(f"Base Pc: {base_pc:.2f} bar → Tuned: {tuned_char.getCriticalPressure():.2f} bar")
print("\nTuning Guidelines:")
print("  Higher Tc → Lower onset pressure")
print("  Higher Pc → Higher onset pressure")

## 7. CPA Equation of State Method

The **Cubic Plus Association (CPA)** EOS extends SRK with association terms:
$$P = P_{cubic} + P_{assoc}$$

### Key CPA Parameters for Asphaltenes
| Parameter | Typical Range | Effect |
|-----------|---------------|--------|
| ε/R | 2500-4500 K | Association strength |
| κ | 0.001-0.05 | Association probability |

In [ ]:
# CPA fluid with asphaltene
cpa_fluid = SystemSrkCPAstatoil(373.15, 300.0)
cpa_fluid.addComponent("methane", 0.35)
cpa_fluid.addComponent("ethane", 0.08)
cpa_fluid.addComponent("propane", 0.05)
cpa_fluid.addComponent("n-heptane", 0.40)
cpa_fluid.addComponent("asphaltene", 0.05)
cpa_fluid.setMixingRule("classic")
cpa_fluid.init(0)
cpa_fluid.init(1)

print(f"CPA fluid created with {cpa_fluid.getNumberOfComponents()} components")
print(f"\nComponent list:")
for i in range(cpa_fluid.getNumberOfComponents()):
    comp = cpa_fluid.getPhase(0).getComponent(i)
    print(f"  {comp.getComponentName()}: z={comp.getz():.4f}")

In [ ]:
# CPA flash calculations
cpa_ops = ThermodynamicOperations(cpa_fluid)

# Pressure scan
pressures_cpa = np.arange(350, 50, -25)
phases_cpa = []

for p in pressures_cpa:
    cpa_fluid.setPressure(float(p))
    try:
        cpa_ops.TPflash()
        phases_cpa.append(cpa_fluid.getNumberOfPhases())
    except:
        phases_cpa.append(1)

plt.figure(figsize=(10, 5))
plt.plot(pressures_cpa, phases_cpa, 'bs-', markersize=8)
plt.xlabel('Pressure [bar]', fontsize=12)
plt.ylabel('Number of Phases', fontsize=12)
plt.title('CPA Phase Behavior - Pressure Depletion', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

## 8. Complete Oil Characterization with TBP Fractions

Realistic oil modeling combines:
- Defined components (C1-C6)
- TBP pseudo-components (C7+)
- Asphaltene pseudo-component

In [ ]:
# Complete oil with TBP fractions
oil = SystemSrkEos(373.15, 200.0)
oil.getCharacterization().setTBPModel("PedersenSRK")

# Light ends
oil.addComponent("nitrogen", 0.005)
oil.addComponent("CO2", 0.02)
oil.addComponent("methane", 0.35)
oil.addComponent("ethane", 0.08)
oil.addComponent("propane", 0.05)
oil.addComponent("n-butane", 0.03)
oil.addComponent("n-pentane", 0.02)

# C7+ TBP fractions: (name, moles, MW in kg/mol, density in g/cm³)
oil.addTBPfraction("C7", 0.10, 0.096, 0.738)
oil.addTBPfraction("C10", 0.08, 0.134, 0.792)
oil.addTBPfraction("C15", 0.05, 0.206, 0.836)
oil.addTBPfraction("C20+", 0.03, 0.350, 0.890)

# Add asphaltene
asph = PedersenAsphalteneCharacterization()
asph.setAsphalteneMW(850.0)
asph.setAsphalteneDensity(1.12)
asph.addAsphalteneToSystem(oil, 0.02)

oil.setMixingRule("classic")
oil.init(0)

print(f"Total components: {oil.getNumberOfComponents()}")
print("\nCharacterized oil ready for flash calculations")

In [ ]:
# Flash and display results
oil_ops = ThermodynamicOperations(oil)
oil_ops.TPflash()

print(f"T = {oil.getTemperature()-273.15:.1f} °C, P = {oil.getPressure():.1f} bar")
print(f"Phases: {oil.getNumberOfPhases()}")

if oil.getNumberOfPhases() >= 1:
    phase = oil.getPhase(0)
    print(f"\nLiquid phase density: {phase.getDensity('kg/m3'):.1f} kg/m³")
    print(f"Liquid phase MW: {phase.getMolarMass()*1000:.1f} g/mol")

## 9. Summary and Best Practices

### Method Selection Guide

| Scenario | Recommended Method |
|----------|-------------------|
| Quick screening | De Boer |
| Routine engineering | Pedersen SRK |
| Detailed design | CPA with tuning |
| Research | CPA + parameter fitting |

### Workflow
1. **Screen** all fluids with De Boer
2. **Analyze** flagged samples with thermodynamic model
3. **Validate** with laboratory AOP tests
4. **Tune** parameters to experimental data
5. **Predict** behavior at field conditions

### References
1. Pedersen, K.S. (2025). SPE-224534-MS, GOTECH Dubai
2. De Boer, R.B. et al. (1995). SPE-24987-PA
3. Li, Z. & Firoozabadi, A. (2010). Energy & Fuels